In [3]:
import pandas as pd

In [4]:
df=pd.read_excel('Top20Sentences_ALLTIERS_NEW_Annotated.xlsx')

In [5]:
df.head()

,Unnamed: 0,Adword,Text,Score,Length of Sentence,Editorial Relevance,Clarity of Sentence,Tier1 Parent
0,0,Automotive,vehicle accident,0.5168,16,Y,N,Automotive
1,1,Automotive,vehicle accident,0.5168,17,Y,N,Automotive
2,2,Automotive,welcome to the royal automobile club,0.5100,36,Y,Y,Automotive
3,3,Automotive,um what kind of car are you driving porsche c...,0.5041,52,Y,Y,Automotive
4,4,Automotive,so you really are a car man,0.5007,27,Y,Y,Automotive


In [6]:
df['Editorial Relevance'].value_counts()

Y    3711
N    1463
Name: Editorial Relevance, dtype: int64

In [7]:
df['Clarity of Sentence'].value_counts()

Y    3534
N    1640
Name: Clarity of Sentence, dtype: int64

In [8]:
dftruth=df[df['Editorial Relevance']=='Y']

In [9]:
#only for Tier 1 Evaluation

dftruth=dftruth[dftruth['Adword']==dftruth['Tier1 Parent']]

In [10]:
dftruth['Editorial Relevance'].value_counts()

Y    1139
Name: Editorial Relevance, dtype: int64

In [11]:
adwords=dftruth['Adword'].to_list()
text=dftruth['Text'].to_list()

In [12]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm  

train_samples = []
for k,v in tqdm(zip(adwords,text)):
    train_samples.append(InputExample(
        texts=[k, v]
    ))

0it [00:00, ?it/s]

In [13]:
print(train_samples[20])

<InputExample> label: 0, texts: Automotive; what the car accident


In [14]:
from sentence_transformers import datasets

#batch_size = 32 for all tiers

batch_size=16

loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size)

In [15]:
from sentence_transformers import models, SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [16]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(model)

In [17]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='./sbert_test_mnr2_Tier1',
    show_progress_bar=True
)  


2022-09-15 03:01:10.829492: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/71 [00:00<?, ?it/s]